# Milestone 1

In [1]:
# Import packages that are used
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Project idea

We want to create an interactive visualization with a map of France and its regions. When hovering over/clicking on a region, one can for example see the wine types grown in that region, their average ratings and typical words used to describe them. 

### Data Cleaning

In [15]:
df = pd.read_csv('Data/winemag-data-130k-v2.csv', sep = ',')
france = df[df.country.str.contains('France', na=False)]
france.head(5)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
7,7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
9,9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam
11,11,France,"This is a dry wine, very spicy, with a tight, ...",NaN,87,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Leon Beyer 2012 Gewurztraminer (Alsace),Gewürztraminer,Leon Beyer
30,30,France,Red cherry fruit comes laced with light tannin...,Nouveau,86,NaN,Beaujolais,Beaujolais-Villages,NaN,Roger Voss,@vossroger,Domaine de la Madone 2012 Nouveau (Beaujolais...,Gamay,Domaine de la Madone
42,42,France,"This is a festive wine, with soft, ripe fruit ...",Nouveau,86,9.0,Beaujolais,Beaujolais,NaN,Roger Voss,@vossroger,Henry Fessy 2012 Nouveau (Beaujolais),Gamay,Henry Fessy


In [3]:
france.isnull().sum()

Unnamed: 0                   0
country                      0
description                  0
designation               7563
points                       0
price                     4317
province                     0
region_1                    76
region_2                 22093
taster_name                265
taster_twitter_handle      265
title                        0
variety                      0
winery                       0
dtype: int64

We are not interested in the columns: 'Unnamed: 0' (it has no meaning), country (we know which country it is), region_2 (it has no values), taster_name & taster_twitter_handle (it's not important who wrote the review).

There are also a lot of missing values for designation and price, but for now we will keep these if they turn out to be of interest later on.

In [4]:
wines = france.drop(['Unnamed: 0', 'country', 'region_2', 'taster_name','taster_twitter_handle'], axis=1)\
    .rename(columns={'region_1':'region'})
wines.head(3)

,description,designation,points,price,province,region,title,variety,winery
7,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
9,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam
11,"This is a dry wine, very spicy, with a tight, ...",NaN,87,30.0,Alsace,Alsace,Leon Beyer 2012 Gewurztraminer (Alsace),Gewürztraminer,Leon Beyer


### Dataset description

The data was scraped from WineEnthusiast on November 22nd, 2017.

| Column name | Description |
| --- | --- |
| description | A few sentences from a sommelier describing the wine's taste, smell, look, feel, etc. | 
| designation | The vineyard within the winery where the grapes that made the wine are from. | 
| points | The number of points WineEnthusiast rated the wine on a scale of 1-100 (though they say they only post reviews for wines that score >=80). | 
| price | The cost for a bottle of the wine. (Currency?) | 
| province | The province or state that the wine is from. |
| region_1 | The wine growing area in a province or state. | 
| title | The title of the wine. | 
| variety | The type of grapes used to make the wine. |
| winery | The winery that made the wine |

### Data Exploration

In [31]:
grouped = wines.groupby(['province', 'region', 'variety'])\
    .agg({'price':'mean', 'points':'mean', 'title':'count'}).reset_index()
grouped

,province,region,variety,price,points,title
0,Alsace,Alsace,Alsace white blend,29.571429,90.235294,51
1,Alsace,Alsace,Auxerrois,24.142857,88.454545,11
2,Alsace,Alsace,Champagne Blend,16.500000,84.000000,2
3,Alsace,Alsace,Chardonnay,23.200000,90.600000,5
4,Alsace,Alsace,Chasselas,22.000000,89.500000,4
...,...,...,...,...,...,...
1032,Southwest France,Vin de Pays du Comté Tolosan,Sauvignon Blanc,10.000000,85.000000,1
1033,Southwest France,Vin de Pays du Comté Tolosan,Syrah,18.000000,89.000000,1
1034,Southwest France,Vin de Pays du Lot,Malbec,12.000000,84.500000,4
1035,Southwest France,Vin de Pays du Lot,Rosé,12.000000,84.000000,1


In [32]:
provinces = wines.groupby(['province', 'region']).agg({'variety':'unique'}).reset_index()
provinces

,province,region,variety
0,Alsace,Alsace,"[Gewürztraminer, Pinot Gris, Riesling, White B..."
1,Alsace,Crémant d'Alsace,"[Sparkling Blend, Pinot Noir, Pinot Gris, Pino..."
2,Beaujolais,Beaujolais,[Gamay]
3,Beaujolais,Beaujolais Blanc,[Chardonnay]
4,Beaujolais,Beaujolais Rosé,"[Rosé, Gamay]"
...,...,...,...
381,Southwest France,Saint-Mont,"[Red Blend, White Blend, Tannat]"
382,Southwest France,Saussignac,[Bordeaux-style White Blend]
383,Southwest France,Vin de Pays des Côtes de Gascogne,"[Ugni Blanc-Colombard, Bordeaux-style Red Blen..."
384,Southwest France,Vin de Pays du Comté Tolosan,"[Chenin Blanc, Malbec, Chardonnay-Viognier, Sa..."


In [33]:
sortedPointsPrice = wines[['title', 'region', 'points', 'price']].sort_values(by=['points', 'price'], ascending=(False,True))
sortedPointsPrice

,title,region,points,price
58352,Château Léoville Barton 2010 Saint-Julien,Saint-Julien,100,150.0
89728,Louis Roederer 2008 Cristal Vintage Brut (Cha...,Champagne,100,250.0
36528,Krug 2002 Brut (Champagne),Champagne,100,259.0
111756,Château Léoville Las Cases 2010 Saint-Julien,Saint-Julien,100,359.0
89729,Salon 2006 Le Mesnil Blanc de Blancs Brut Char...,Champagne,100,617.0
...,...,...,...,...
90932,Château Timberlay 2009 Bordeaux Supérieur,Bordeaux Supérieur,80,20.0
123543,La Cave des Vignerons de Pfaffenheim 2009 Pino...,Alsace,80,20.0
92450,Michel Picard 2007 Sancerre,Sancerre,80,23.0
79714,Domaine Bernhard & Reibel 2014 Hahnenberg Pino...,Alsace,80,25.0


In [56]:
bestValue = wines[['title', 'region', 'points', 'price']]
bestValue['points/price'] = wines['points'] / wines['price']
bestValue = bestValue.sort_values(by=['points/price'], ascending=False)
bestValue

C:\Users\Thomas\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,region,points,price,points/price
122407,Belle Made For You NV White (Vin de France),Vin de France,84,5.0,16.800000
5789,Belle Made For You NV Cabernet Sauvignon (Vin ...,Vin de France,83,5.0,16.600000
105918,French Rabbit 2007 Pinot Noir (Vin de Pays d'Oc),Vin de Pays d'Oc,83,5.0,16.600000
5801,Belle Made For You NV Red (Vin de France),Vin de France,83,5.0,16.600000
119016,Le Pépin 2015 Moscato (Vin de France),Vin de France,85,6.0,14.166667
...,...,...,...,...,...
129758,Les Maîtres Vignerons de la Presqu'île de Sain...,Côtes de Provence,87,NaN,NaN
129759,Les Vignobles Gueissard 2013 Cuvée G Rosé (Côt...,Côtes de Provence,87,NaN,NaN
129764,Pierre Sparr 2012 Pinot Gris (Alsace),Alsace,87,NaN,NaN
129787,Domaine Poulleau Père et Fils 2010 Volnay,Volnay,89,NaN,NaN
